# Ejercicio con kNNs

Primero, descargamos un dataset sobre el que vamos a trabajar en nuestro ejercicio en un subdirectorio `data` de nuestro directorio de trabajo:

In [33]:
!curl -o data/diabetes.csv https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv

% Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23873  100 23873    0     0  26853      0 --:--:-- --:--:-- --:--:-- 26823


A continuación, cargamos los módulos que utilizaremos en el ejercicio:

In [2]:
# Nuestros sospechosos habituales
import pandas as pd
import numpy as np

# También, como siempre, nos apoyamos en Scikit-Learn para hacer el split en training y test
from sklearn.model_selection import train_test_split
# Usaremos el preprocesador StandardScaler para no tener sesgos en los datos
# de entrada
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
# Importamos herramientas para evaluar el modelo. F1 es la media
# armónica de precision y recall
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

Ahora es el turno de cargar el dataset en un Pandas dataframe y de investigar un poco qué es lo que tenemos en términos de número de muestras y de features. Apoyaos en la documentación pública de Pandas para investigar cómo cargar un dataset (lo veremos en clase de todas formas). Escribid el código en los placeholders marcados con `#tu código aquí#`:

In [14]:
# Leemos el CSV que hemos descargado con la ayuda de los métos que nos ofrecen
# los dataframes de Pandas
dataset = pd.read_csv("data/diabetes.csv")

# Vemos cuántas muestras tiene nuestro dataset
print(f"El dataset tiene {dataset.shape[0]} muestras")

# Miramos también qué pinta tiene el dataset desde el punto de vista de características y etiquetas, usando Pandas
dataset.head()
#Vemos las estadísticas con describe
dataset.describe()
#Vemos si tiene NaN
dataset.isna().sum()

El dataset tiene 768 muestras


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

Trata de investigar si hay columnas del dataset que contienen valores que son nulos y que no tienen sentido. Como ejemplo, un grosor de piel de valor cero no tiene mucho sentido:

In [16]:
# Escribe código para filtrar los valores nulos de 1 columna del DataFrame y visualizarlos,
dataset[dataset["SkinThickness"] <= 0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
2,8,183,64,0,0,23.3,0.672,32,1
5,5,116,74,0,0,25.6,0.201,30,0
7,10,115,0,0,0,35.3,0.134,29,0
9,8,125,96,0,0,0.0,0.232,54,1
10,4,110,92,0,0,37.6,0.191,30,0
...,...,...,...,...,...,...,...,...,...
757,0,123,72,0,0,36.3,0.258,52,1
758,1,106,76,0,0,37.5,0.197,26,0
759,6,190,92,0,0,35.5,0.278,66,1
762,9,89,62,0,0,22.5,0.142,33,0


Ahora, para la columna correspondiente a la feature que has elegido explorar (por ejemplo, skinThickness), calcula la media de los valores:

In [17]:
# Calcula la media de los valores de 1 columna de tu DataFrame
dataset["SkinThickness"].mean()

20.536458333333332

Lo que vamos a hacer ahora es sustituir los valores nulos por la media de todos los valores de la columna, con el objetivo de poder seguir contando con las muestras que los tienen. Como pista, debes hacer esa operación para todas estas columnas en el DataFrame: 'Glucose','BloodPressure','SkinThickness','BMI','Insulin'. Lo mejor es que escribas código que itere sobre esas columnas para realizar la operación.

Recordar que este ejercicio se puede resolver de manera simple con `sklearn.Impute.SimpleImputer`

In [95]:
# Almacena las columnas a alterar en una lista de Python
col_names = list(dataset.columns)
means = list(dataset.mean())

values = {names:mean for names, mean in zip(col_names, means)}

#Pero quiero que la variable y sea 0
values["Outcome"] = 0
values

# Itera sobre cada elemento de la lista, calculando la media y posteriormente sustituyendo los ceros por el valor calculado de la media
dataset_c = dataset.copy()

for k in dataset_c.columns:
    dataset_c[k] = dataset_c[k].replace(0,values.get(k))

In [97]:
dataset_c

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.000000,148.0,72.0,35.000000,79.799479,33.6,0.627,50,1
1,1.000000,85.0,66.0,29.000000,79.799479,26.6,0.351,31,0
2,8.000000,183.0,64.0,20.536458,79.799479,23.3,0.672,32,1
3,1.000000,89.0,66.0,23.000000,94.000000,28.1,0.167,21,0
4,3.845052,137.0,40.0,35.000000,168.000000,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10.000000,101.0,76.0,48.000000,180.000000,32.9,0.171,63,0
764,2.000000,122.0,70.0,27.000000,79.799479,36.8,0.340,27,0
765,5.000000,121.0,72.0,23.000000,112.000000,26.2,0.245,30,0
766,1.000000,126.0,60.0,20.536458,79.799479,30.1,0.349,47,1


In [98]:
dataset.mean()

Pregnancies                   3.845052
Glucose                     120.894531
BloodPressure                69.105469
SkinThickness                20.536458
Insulin                      79.799479
BMI                          31.992578
DiabetesPedigreeFunction      0.471876
Age                          33.240885
Outcome                       0.348958
dtype: float64

In [99]:
dataset_c.mean()

Pregnancies                   4.400782
Glucose                     121.681605
BloodPressure                72.254807
SkinThickness                26.606479
Insulin                     118.660163
BMI                          32.450805
DiabetesPedigreeFunction      0.471876
Age                          33.240885
Outcome                       0.348958
dtype: float64

Antes de comenzar con el proceso de entrenamiento, como siempre, partimos el dataset en training y test. Elige un 20% del dataset para el conjunto de testing, dejando el 80% restante para el entrenamiento:

In [100]:
y = dataset_c.loc[:,"Outcome"]
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [102]:
from sklearn.model_selection import train_test_split

# Recuerda que la columna nueve (índice 8) es la que tiene nuestras etiquetas, y que el resto contiene las features.
X = dataset_c.iloc[:,0:7]
y = dataset_c.loc[:,"Outcome"]
# Haz el split, selecciona un 20% del dataset original como datos de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=0)

Lo siguiente será escalar los datos, ésta es una técnica de ingeniería de características que será necesaria para poder calcular distancias correctamente con KNNs (veremos más sobre esto en clase):

In [103]:
# Ahora escalamos los datos, de manera que todos los rangos van desde -1 hasta 1.
sc_X = StandardScaler()
# Hacemos training y transformación conjunta sobre el training set
X_train = sc_X.fit_transform(X_train)
# Tenemos que asegurarnos de que el testing set también está transformado
X_test = sc_X.transform(X_test)

Una buena estimación del número de vecinos suele ser la raíz cuadrada del número de características. Usa este número como tu configuración inicial de `n_neighbors` y procede crear y entrenar un kNN:

In [115]:
n_neighbors = round(math.sqrt(len(dataset_c.columns)-1))
n_neighbors

3

In [119]:
import math
from sklearn.neighbors import KNeighborsRegressor

# Define el modelo, inicializando kNN con los datos seleccionados
n_neighbors = round(math.sqrt(len(dataset_c.columns)-1))
 
cls = KNeighborsRegressor(n_neighbors = n_neighbors, metric="euclidean")
# Entrena el modelo
cls.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='euclidean',
                    metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                    weights='uniform')

Finalmente, evalua el modelo sobre el dataset de test, y muestra la precisión obtenida:

In [120]:
predictions = cls.predict(X_test)
print("Precisión sobre el juego de pruebas: {:.2f}"
      .format(cls.score(X_test, y_test)))

Precisión sobre el juego de pruebas: 0.11
